In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.onnx as onnx
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [2]:
import openpyxl
import pprint as pp
import xlrd
import csv
import pickle

In [3]:
def csv_from_excel():
    wb = xlrd.open_workbook('/dat/ie_data.xls')
    sh = wb.sheet_by_name('Data')
    your_csv_file = open('/dat/pe_shill.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)
    
    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))
    your_csv_file.close()

# shiller csv, good data has 22 elements. Still need to format labels
def csv_to_arr(arr=[]):
    
    pe_array = arr
    with open('pe_shill.csv') as file:
        reader = csv.reader(file, quotechar='|')
        for row in reader:
            pe_array.append(row)

    pe_array = pe_array[5:]
    return pe_array

arr = csv_to_arr() 
label_arr = arr[:3]
arr = arr[3:]

labels = []
count = 0
for x in label_arr:
    
    curr = 0
    for y in x:
        if len(labels) <= curr and count == 0:
            labels.append(y.replace('"',' '))
            curr += 1
        else:
            labels[curr] = labels[curr] + y.replace('"',' ')
            curr += 1
    count += 1

#print(len(labels))
#pp.pprint(labels)
#print(arr)

def clean_arr(arr):
    cleaned = []
    curr = 0
    for i in arr:
        cleaned.append([])
        
        for j in i:
            
            foo = j[1:-1]
            try:
                cleaned[curr].append([float(foo)])
            except:            
                cleaned[curr].append([0])
        curr += 1
    return cleaned

pe_arr = clean_arr(arr)
arr = 0
pe_arr = np.asarray(pe_arr)

x_pe = torch.from_numpy(pe_arr)
pe_arr = 0
x_pe = x_pe.float()
x_pe.size()

torch.Size([1804, 22, 1])

In [4]:
device = torch.device("cuda:0")

class GNet(nn.Module):
    def __init__(self):
        super(GNet, self).__init__()

        self.ker = 1
        self.ngf = 1804
        self.main = nn.Sequential(
            nn.ConvTranspose1d(100, self.ngf*8, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ngf*8),
            nn.ReLU(True),
            
            nn.ConvTranspose1d(self.ngf*8, self.ngf*4, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ngf*4),
            nn.ReLU(True),
            
            nn.ConvTranspose1d(self.ngf*4, self.ngf*2, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ngf*2),
            nn.ReLU(True),
            
            nn.ConvTranspose1d(self.ngf*2, self.ngf, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ngf),
            nn.ReLU(True),
            
            nn.ConvTranspose1d(self.ngf, 22, self.ker, 2, bias=False),
            nn.Tanh()
        )
        
    def forward(self, input):
        
        return self.main(input)
    

class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()

        self.ker = 1
        self.ndf = 1804
        self.main = nn.Sequential(
            nn.Conv1d(22, self.ndf, self.ker, 2, bias=False), 
            nn.LeakyReLU(0.2, inplace=True), 
            
            nn.Conv1d(self.ndf, self.ndf*2, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ndf*2), 
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv1d(self.ndf*2, self.ndf*4, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ndf*4), 
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv1d(self.ndf*4, self.ndf*8, self.ker, 2, bias=False),
            nn.BatchNorm1d(self.ndf*8), 
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv1d(self.ndf*8, 1, self.ker, 2, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, input):
        return self.main(input)
        

In [5]:
G = GNet().to(device)
D = DNet().to(device)

In [6]:
print(G)
print(D)

GNet(
  (main): Sequential(
    (0): ConvTranspose1d(100, 14432, kernel_size=(1,), stride=(2,), bias=False)
    (1): BatchNorm1d(14432, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose1d(14432, 7216, kernel_size=(1,), stride=(2,), bias=False)
    (4): BatchNorm1d(7216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose1d(7216, 3608, kernel_size=(1,), stride=(2,), bias=False)
    (7): BatchNorm1d(3608, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose1d(3608, 1804, kernel_size=(1,), stride=(2,), bias=False)
    (10): BatchNorm1d(1804, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose1d(1804, 22, kernel_size=(1,), stride=(2,), bias=False)
    (13): Tanh()
  )
)
DNet(
  (main): Sequential(
    (0): Conv1d(22, 1804, kernel_size=(1,)

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

G.apply(weights_init)
D.apply(weights_init)

criterion = nn.BCELoss()

fixed_noise = torch.randn(100, 14432, 1, device=device)

real_label = 1
fake_label = 0
optimizerD = optim.Adam(D.parameters(), lr=.0002, betas=(.5, 0.999))
optimizerG = optim.Adam(G.parameters(), lr=.0002, betas=(.5, 0.999))

ds = torch.utils.data.TensorDataset(x_pe)
x_pe = 0
loader = DataLoader(ds, batch_size=1804, shuffle=True, num_workers=2)


In [ ]:
G_losses = []
D_losses = []

print("Start Training...")
for epoch in range(1000):
    for i, data in enumerate(loader, 0):

        D.zero_grad()
        
        real = data[0].to(device)
        b_size = real.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        output = D(real).view(-1)
        errD_real = criterion(output, label)
        
        errD_real.backward()
        D_x = output.mean().item()
        
        # Train w/ fake
        noise = torch.randn(b_size, 100, 1, device=device)
        
        fake = G(noise)
        label.fill_(fake_label)
        
        output = D(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        
        errD = errD_real + errD_fake
        
        optimizerD.step()
        
        # update G
        G.zero_grad()
        label.fill_(real_label)
        
        output = D(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()
        
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, 1000, i, len(loader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
torch.save(D, '.')
torch.save(G, '.')

Start Training...
[0/1000][0/1]	Loss_D: 1.8980	Loss_G: 6.8640	D(x): 0.4534	D(G(z)): 0.4931 / 0.5111


In [ ]:
torch.save({
            'epoch': 1,
            'model_state_dict': D.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': D_losses,
            }, 'D.p')

torch.save({
            'epoch': 1,
            'model_state_dict': G.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': G_losses,
            }, 'G.p')

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('test_1.png')
plt.show()
